# Assignment 6: Mustaeen Ahmed

In [ ]:
import pygame
import moderngl
import glm
from LoadObject import getObjectData

In [ ]:
pygame.init()

WIDTH = 840
HEIGHT = 480

clock = pygame.time.Clock()
running = True

pygame.display.gl_set_attribute(pygame.GL_CONTEXT_MAJOR_VERSION, 3)
pygame.display.gl_set_attribute(pygame.GL_CONTEXT_MINOR_VERSION, 3)
pygame.display.gl_set_attribute(pygame.GL_CONTEXT_PROFILE_MASK, pygame.GL_CONTEXT_PROFILE_CORE)
pygame.display.gl_set_attribute(pygame.GL_CONTEXT_FORWARD_COMPATIBLE_FLAG, True)

In [ ]:
pygame.display.gl_set_attribute(pygame.GL_CONTEXT_PROFILE_MASK, pygame.GL_CONTEXT_PROFILE_CORE)
pygame.display.set_mode((WIDTH, HEIGHT), flags=pygame.OPENGL | pygame.DOUBLEBUF | pygame.RESIZABLE)
pygame.display.set_caption(title="Assignment 06: Mustaeen Ahmed")
gl = moderngl.get_context()
gl.info["GL_VERSION"]


In [ ]:
geo_data, bound = getObjectData("teapot_with_texCoords.obj", texture=True)
geo_buffer = gl.buffer(geo_data)

In [ ]:
face_vertex_shader_code = '''
    #version 330 core

    layout (location=0) in vec3 position;
    layout (location=1) in vec2 uv;

    out vec2 f_uv;
    uniform mat4 view, perspective;

    void main() {
        f_uv = uv;
        gl_Position = perspective * view * vec4(position, 1.0);
    }
'''

face_fragment_shader_code = '''
    #version 330 core
    in vec2 f_uv;
    uniform sampler2D map;

    out vec4 color;

    void main() {
        color = texture(map, f_uv);
    }
'''

In [ ]:
face_program = gl.program(
    vertex_shader=face_vertex_shader_code,
    fragment_shader=face_fragment_shader_code
)

face_renderable = gl.vertex_array(face_program,
    [(geo_buffer, "3f 2f", "position", "uv")]
)

In [ ]:
texture_img = pygame.image.load("brick.jpg")
texture_data = pygame.image.tobytes(texture_img, "RGB", True)
face_texture = gl.texture(texture_img.get_size(), data = texture_data, components = 3)

face_texture.build_mipmaps()

face_sampler = gl.sampler(texture=face_texture, filter=(gl.LINEAR_MIPMAP_LINEAR, gl.LINEAR))

In [ ]:
displacement = 2 * bound.radius * glm.rotate(glm.vec3(0,1,0), glm.radians(60), glm.vec3(1,0,0))
target_point = glm.vec3(bound.center)
up_vector = glm.vec3(0,1,0)

fov = glm.radians(45)
aspect = WIDTH/HEIGHT
near = bound.radius
far = 3 * bound.radius

perspective_matrix = glm.perspective(fov, WIDTH/HEIGHT, near, far)

In [ ]:
running = True
clock = pygame.time.Clock()
alpha = 0
gl.enable(gl.DEPTH_TEST)

while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT or (event.type == pygame.KEYDOWN and event.key == 27):
            running = False
        elif (event.type == pygame.WINDOWRESIZED):
            width = event.x
            height = event.y
            perspective_matrix = glm.perspective(fov, WIDTH / HEIGHT, near, far)

    eye_point = glm.vec3(bound.center) + glm.rotate(displacement, glm.radians(alpha), glm.vec3(0, 1, 0))
    view_matrix = glm.lookAt(eye_point, target_point, up_vector)

    gl.clear(0.0, 0.0, 0.0)

    face_program['view'].write(view_matrix)
    face_program['perspective'].write(perspective_matrix)

    face_sampler.use(0)
    face_program['map'] = 0
    face_renderable.render(moderngl.TRIANGLES)

    pygame.display.flip()
    clock.tick(60)

    alpha = alpha + 1
    if alpha > 360:
        alpha = 0

pygame.display.quit()